# Proyecto: Marketing Modeling Mix

Enlace a la data:
https://www.kaggle.com/harrimansaragih/dummy-advertising-and-sales-data

## Carga de datos

- TV: Presupuesto de promoción en TV (en Millones)
- Radio: Presupuesto de promoción en Radio (en Millones)
- Social Media: Presupuesto de promoción en redes sociales (en Millones)
- Influencer: Tipo de infulencer con el que se colaboró (Mega, Macro, Nano, o Micro influencer)
- Sales: Ventas (en Millones)

In [1]:
import numpy as np
import pandas as pd
import altair as alt

In [2]:
path = 'data/dataset.csv'

In [3]:
data = pd.read_csv(path)
data.head()

,TV,Radio,Social Media,Influencer,Sales
0,16.0,6.566231,2.907983,Mega,54.732757
1,13.0,9.237765,2.409567,Mega,46.677897
2,41.0,15.886446,2.913410,Mega,150.177829
3,83.0,30.020028,6.922304,Mega,298.246340
4,15.0,8.437408,1.405998,Micro,56.594181


## EDA

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4572 entries, 0 to 4571
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   TV            4562 non-null   float64
 1   Radio         4568 non-null   float64
 2   Social Media  4566 non-null   float64
 3   Influencer    4572 non-null   object 
 4   Sales         4566 non-null   float64
dtypes: float64(4), object(1)
memory usage: 178.7+ KB


Los tipos de datos son correctos. Pero, hay valores faltantes en TV, Radio, Social Media y Sales

Se calcula las estadisticas descriptivas del dataset.

In [5]:
data.describe()

,TV,Radio,Social Media,Sales
count,4562.000000,4568.000000,4566.000000,4566.000000
mean,54.066857,18.160356,3.323956,192.466602
std,26.125054,9.676958,2.212670,93.133092
min,10.000000,0.000684,0.000031,31.199409
25%,32.000000,10.525957,1.527849,112.322882
50%,53.000000,17.859513,3.055565,189.231172
75%,77.000000,25.649730,4.807558,272.507922
max,100.000000,48.871161,13.981662,364.079751


In [6]:
data.isnull().sum()

TV              10
Radio            4
Social Media     6
Influencer       0
Sales            6
dtype: int64

- TV → 10 valores nulos
- Radio → 4 valores nulos
- Social Media → 6 valores nulos
- Sales → 6 valores nulos
<p>
    El porcentaje de valores nulos de cada columna con respecto al total de datos es inferior a 1%.
</p>

In [7]:
data['TV'].fillna(data['TV'].median(), inplace=True)
data['Radio'].fillna(data['Radio'].median(), inplace=True)
data['Social Media'].fillna(data['Social Media'].median(), inplace=True)
data.dropna(inplace=True)

→ Relleno los valores de los features porque si tienen ventas, se perderían datos útiles para entrenar el modelo. </br>
→ Solamente elimino las filas de las ventas vacías para que el Y obtenido sea el original de la data.

Obtengo los valores únicos de "Influencer" para ver posibles errores de tipeo

In [8]:
data.Influencer.unique()

array(['Mega', 'Micro', 'Nano', 'Macro'], dtype=object)

Se visualiza el conteo del campo "Influencer"

In [9]:
data['Influencer'].value_counts()

Mega     1156
Micro    1152
Nano     1137
Macro    1121
Name: Influencer, dtype: int64

- En general, se colaboró con más frecuencia con los mega y micro influencers.

In [10]:
table = pd.pivot_table(data, values=['TV','Social Media', 'Radio'], index='Influencer', aggfunc=np.sum)
table = table.reset_index()
table['Sales'] = data['Sales']
table['Influencer'] = table.apply(lambda x: str(x['Influencer'] + 'influencer'), axis=1)
table

,Influencer,Radio,Social Media,TV,Sales
0,Macroinfluencer,20549.929548,3779.259043,61731.0,54.732757
1,Megainfluencer,20886.982058,3899.938950,61833.0,46.677897
2,Microinfluencer,20994.955118,3773.797590,62011.0,150.177829
3,Nanoinfluencer,20492.870330,3728.177547,61289.0,298.246340


### Visualizaciones exploratorias

In [11]:
alt.Chart(data).mark_circle(size=40,stroke='#000000', strokeWidth=0.2).encode(
    alt.X(alt.repeat("column"), type='quantitative'),
    alt.Y(alt.repeat("row"), type='quantitative'),
    color='Influencer:N',
    tooltip=[alt.Tooltip('Sales', title='Ventas generadas',format='~s')]
).properties(
    width=230,
    height=230
).repeat(
    row=['Sales'],
    column=['Radio', 'TV', 'Social Media']
).interactive()

alt.RepeatChart(...)

Los 3 gráficos tienen diferentes desviaciones. Pero, el de TV tiene valores atípicos para un valor entre 40 y 60.

In [12]:
alt.Chart(table).mark_bar().encode(
    x=alt.X('Influencer:O', title='', axis=None),
    y=alt.Y('Sales:Q',title='Ventas en Millones', stack=None, axis=alt.Axis(titlePadding=60, titleAngle=0, titleAnchor='middle' ,labelFontSize=12, grid=False)),
    color="Influencer:N",
    tooltip = [alt.Tooltip('Influencer:O'),
               alt.Tooltip('Sales:Q')],
).properties(
    title = 'Ventas totales por tipo de influencer',
    width = 400,
    height = 300
).configure_title(
    fontSize = 18,
    anchor = 'middle'
).configure_view(
    strokeWidth=0
)

alt.Chart(...)

A nivel global, cuando se colabora con nanoinfluencers y microinfluencers, se generan más ventas.

In [13]:
base = alt.Chart(data)

bar = base.mark_bar(color='#1A9873').encode(
    x=alt.X('Sales:Q', title='Ventas en Millones y Promedio global' ,bin=True),
    y=alt.Y('count()', title='Frecuencia relativa', axis=alt.Axis(titlePadding=20)),
    tooltip=['count()']
).properties(
    title='Histograma de las ventas',
    width = 600,
    height = 200
)

rule = base.mark_rule(color='red').encode(
    x='mean(Sales):Q',
    size=alt.value(5)
)

bar + rule

alt.LayerChart(...)

La distribución de las ventas registradas es casi simétrica y no presenta valores atípicos. Además, sólo se presenta gran dispersión en los extremos de la distribución y la media resulta representativa para la mayor proporción de los datos.

In [14]:
corr_table = data.corr()
corr_table = corr_table.reset_index()
corr_table = corr_table.melt('index')
corr_table.columns = ['variable1','variable2','correlacion']
corr_table['corrRedondeado'] = corr_table.apply(lambda x: round(x['correlacion'], 2), axis=1 )

In [15]:
base = alt.Chart(corr_table).mark_rect().encode(
    x=alt.X('variable1:O',title='',axis=alt.Axis(labelFontSize=12, labelFontWeight='bold')),
    y=alt.Y('variable2:O',title='',axis=alt.Axis(labelFontSize=12, labelFontWeight='bold')),
    color=alt.Color('correlacion:Q', scale=alt.Scale(scheme='purples')),
    tooltip=['variable1','variable2','correlacion']
).properties(
    width=300,
    height=300
)
text = base.mark_text().encode(
    text='corrRedondeado',
    color=alt.condition(
        alt.datum.corrRedondeado > 0.8, 
        alt.value('white'),
        alt.value('black')
    )
)

base + text

alt.LayerChart(...)

El presupuesto destinado para el canal de marketing "TV" está fuertemente relacionado a las ventas.

## Modelo de regresión lineal múltiple

$Y_{Sales} = a + b*(X_{TV}) + c*(X_{Radio}) + d*(X_{Redes Sociales})$

In [16]:
from sklearn.linear_model import LinearRegression

X = data[['TV','Radio','Social Media']]
y = data[['Sales']]

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=147)

In [18]:
lr = LinearRegression()
lr.fit(X_train,y_train)

LinearRegression()

In [19]:
print(f"La intersección del modelo lineal:\n a: {lr.intercept_[0]}")
print(f"Los coeficientes del modelo lineal:\n b: {lr.coef_[0,0]}\n c: {lr.coef_[0,1]}\n d: {lr.coef_[0,2]}")

La intersección del modelo lineal:
 a: -0.2706857715403146
Los coeficientes del modelo lineal:
 b: 3.5109014701835886
 c: 0.15151762282660808
 d: 0.055322866982557554


$Y_{Sales} = (-0.271) + (3.51)*(X_{TV}) + (0.152)*(X_{Radio}) + (0.0553)*(X_{Redes Sociales})$

In [20]:
from sklearn import metrics
def print_metrics(true, predicted):
    print('-'*50)
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    print('MAE:', mae)
    print('MSE:', mse)
    print('RMSE:', rmse)

In [21]:
train_pred = lr.predict(X_train)
test_pred = lr.predict(X_test)

In [22]:
print('EVALUANDO EL SET DE PRUEBA')
print_metrics(y_test, test_pred)

EVALUANDO EL SET DE PRUEBA
--------------------------------------------------
MAE: 2.625162447103428
MSE: 33.73045693667616
RMSE: 5.807792776664485


In [23]:
print('EVALUANDO EL SET DE ENTRENAMIENTO')
print_metrics(y_train, train_pred)

EVALUANDO EL SET DE ENTRENAMIENTO
--------------------------------------------------
MAE: 2.7232783835992977
MSE: 44.21152938389566
RMSE: 6.649175090482703


In [24]:
df_results_train = pd.concat([X_train, y_train], axis=1)
df_results_train['Sales Predict'] = train_pred[:]
df_results_train['AbsoluteDifference'] = df_results_train.apply(lambda x: abs(x['Sales Predict']-x['Sales']), axis=1)

In [25]:
df_results_test = pd.concat([X_test, y_test], axis=1)
df_results_test['Sales Predict'] = test_pred[:]
df_results_test['AbsoluteDifference'] = df_results_test.apply(lambda x: abs(x['Sales Predict']-x['Sales']), axis=1)

In [26]:
alt.Chart(df_results_train).mark_bar(color="#1A9873").encode(
    x='Sales:Q',
    y='AbsoluteDifference:Q',
    tooltip=['AbsoluteDifference:Q']
).properties(
    title='Histograma del error absoluto de las ventas en el set de entrenamiento',
    width = 800,
    height = 300
)

alt.Chart(...)

→ En los valores de los extremos de la distribución de las ventas, la regresión funionó muy bien en el set de entrenamiento.</br>
→ Se puede apreciar que hay 7 picos en los que el error absoluto es exageradamente grande(superior a 20.05).

### Visualizaciones para el conjunto de entrenamiento

In [27]:
alt.Chart(df_results_train).mark_circle(size=40,stroke='#000000', fill='#92DAFA',strokeWidth=0.2).encode(
    alt.X(alt.repeat("column"), type='quantitative'),
    alt.Y(alt.repeat("row"), type='quantitative'),
    tooltip=['Sales']
).properties(
    width=230,
    height=230
).repeat(
    row=['Sales','Sales Predict'],
    column=['Radio', 'TV', 'Social Media']
).interactive()

alt.RepeatChart(...)

→ El algotritmo tiene errores absolutos muy grandes cuando el presupuesto de TV es de 53 millones, porque en ese presupuesto en particular, hay</br> **valores atípicos**. Todo lo mencionado es visualizable en las gráficas presentadas.

In [28]:
alt.Chart(df_results_train[df_results_train['AbsoluteDifference']<20.05]).mark_bar(color="#F88520").encode(
    x='Sales:Q',
    y='AbsoluteDifference:Q',
    tooltip=['AbsoluteDifference:Q']
).properties(
    title='Histograma del error absoluto de las ventas en el set de entrenamiento (SIN ATÍPICOS)',
    width = 800,
    height = 300
)

alt.Chart(...)

### Visualizaciones para el conjunto de prueba

In [29]:
alt.Chart(df_results_test).mark_bar(color="#6F92B9").encode(
    x='Sales:Q',
    y='AbsoluteDifference:Q',
    tooltip=['AbsoluteDifference:Q']
).properties(
    title='Histograma del error absoluto de las ventas en el set de prueba',
    width = 800,
    height = 300
)

alt.Chart(...)

→ En los valores de los extremos de la distribución de las ventas, la regresión funionó muy bien en el set de prueba.</br>
→ Se puede apreciar que hay 2 picos en los que el error absoluto es exageradamente grande(superior a 120).

In [30]:
alt.Chart(df_results_test).mark_circle(size=40,stroke='#000000', fill='#AF78EA',strokeWidth=0.2).encode(
    alt.X(alt.repeat("column"), type='quantitative'),
    alt.Y(alt.repeat("row"), type='quantitative'),
    tooltip=['Sales']
).properties(
    width=230,
    height=230
).repeat(
    row=['Sales','Sales Predict'],
    column=['Radio', 'TV', 'Social Media']
).interactive()

alt.RepeatChart(...)